# Template System & Phonon Workflows

Learn to use the CrystalMath template system and PhononWorkflow for vibrational calculations.

**Prerequisites:**
- Run `uv sync` in the crystalmath repository root
- This notebook uses real workflow APIs

**What you'll learn:**
1. Browse and filter available DFT templates
2. Load templates for different DFT codes (CRYSTAL, VASP, QE)
3. Configure phonon workflows with phonopy
4. Generate displacement structures
5. Analyze phonon results (frequencies, thermal properties)
6. Combine templates with workflows for end-to-end calculations

## 1. Imports

In [ ]:
from crystalmath.templates import list_templates, load_template, get_template, TemplateInfo
from crystalmath.workflows.phonon import (
    PhononWorkflow,
    PhononConfig,
    PhononMethod,
    PhononDFTCode,
    PhononResult,
)
from crystalmath.api import CrystalController
from crystalmath.models import JobSubmission, DftCode

print("Imports successful!")


## 2. Browse Available Templates

The template system organizes DFT input templates by category and code.

In [ ]:
# List all templates
all_templates = list(list_templates())
print(f"Total templates: {len(all_templates)}\n")

# Group by category
categories = {}
for t in all_templates:
    categories.setdefault(t.category, []).append(t)

for cat, templates in sorted(categories.items()):
    print(f"--- {cat} ({len(templates)} templates) ---")
    for t in templates:
        desc = t.description.split('\n')[0][:60] if t.description else ""
        print(f"  {t.name}: {desc}")
    print()


## 3. Filter Templates

You can filter templates by category or DFT code.

In [ ]:
# Filter by category
print("Advanced templates:")
for t in list_templates(category="advanced"):
    desc_short = t.description.split('\n')[0][:70] if t.description else ''
    print(f"  {t.category}/{t.name}: {desc_short}")

print("\nVASP templates:")
for t in list_templates(dft_code="vasp"):
    print(f"  {t.category}/{t.name}")


## 4. Load a Template

Load template content using the `category/name` identifier.

In [ ]:
# Load a specific template by ID (category/name)
template_id = "advanced/band_structure"
data = load_template(template_id)

if data:
    print(f"Template '{template_id}' loaded successfully!")
    print(f"Keys: {list(data.keys())}")
    # Template contents vary - show what's available
    for key, value in data.items():
        if isinstance(value, str) and len(value) > 100:
            print(f"  {key}: {value[:100]}...")
        else:
            print(f"  {key}: {value}")
else:
    print(f"Template '{template_id}' not found")


In [ ]:
# Get the file path to a template
path = get_template("advanced/band_structure")
if path:
    print(f"Template file: {path}")
    print(f"Exists: {path.exists()}")


## 5. Phonon Workflow Configuration

Configure a phonon calculation using the phonopy finite displacement method.

In [ ]:
# Configure a phonon calculation
config = PhononConfig(
    source_job_pk=1,  # PK of a completed SCF job
    method=PhononMethod.PHONOPY,
    supercell_dim=[2, 2, 2],
    displacement_distance=0.01,  # Angstrom
    use_symmetry=True,
    mesh=[20, 20, 20],
    band_path="AUTO",
    compute_thermal=True,
    tmin=0.0,
    tmax=1000.0,
    tstep=10.0,
    dft_code=PhononDFTCode.CRYSTAL,
    name_prefix="si_phonon",
)

workflow = PhononWorkflow(config)

print("Phonon workflow configured:")
print(f"  Method: {config.method.value}")
print(f"  Supercell: {config.supercell_dim}")
print(f"  Displacement: {config.displacement_distance} A")
print(f"  Symmetry: {config.use_symmetry}")
print(f"  Thermal: {config.compute_thermal} ({config.tmin}-{config.tmax} K)")


## 6. Phonon Methods

Multiple phonon calculation methods are supported.

In [ ]:
print("Available phonon methods:")
for method in PhononMethod:
    print(f"  {method.value}")

print("\nPhonon DFT codes:")
for code in PhononDFTCode:
    print(f"  {code.value}")


## 7. Generate Displacements

The workflow generates displaced structures for finite difference calculations.

In [ ]:
# Example structure (silicon diamond)
demo_cell = [[3.867, 0, 0], [0, 3.867, 0], [0, 0, 3.867]]
demo_positions = [[0, 0, 0], [0.25, 0.25, 0.25]]
demo_symbols = ["Si", "Si"]

# Generate displacements
displacements = workflow.generate_displacements_phonopy(
    demo_cell, demo_positions, demo_symbols
)

print(f"Generated {len(displacements)} displacement structures")
print(f"Total atoms: {len(demo_positions)}")
print(f"Displacements per atom: {len(displacements) // len(demo_positions)}")
print(f"\nFirst displacement:")
print(f"  Atom: {displacements[0]['atom_index']} ({displacements[0]['atom_symbol']})")
print(f"  Direction: {displacements[0]['direction']}")
print(f"  Amplitude: {displacements[0]['amplitude']} A")


## 8. Understanding Phonon Results

The `PhononResult` dataclass tracks the workflow status and computed properties.

In [ ]:
# The PhononResult dataclass
result = PhononResult(
    status="completed",
    n_displacements=12,
    force_sets_ready=True,
    has_imaginary=False,
    min_frequency=120.5,  # cm^-1
    zero_point_energy_ev=0.05,
)

print("PhononResult fields:")
print(f"  status: {result.status}")
print(f"  n_displacements: {result.n_displacements}")
print(f"  force_sets_ready: {result.force_sets_ready}")
print(f"  has_imaginary: {result.has_imaginary}")
print(f"  min_frequency: {result.min_frequency} cm^-1")
print(f"  zero_point_energy_ev: {result.zero_point_energy_ev} eV")
print(f"  completed_count: {result.completed_count}")
print(f"  failed_count: {result.failed_count}")
print(f"\nAs dict:")
print(result.to_dict())


## 9. CRYSTAL Phonon Input Generation

For CRYSTAL, the workflow can generate native phonon keywords instead of using phonopy.

In [ ]:
# Generate CRYSTAL FREQCALC input (Gamma point only)
freq_input = workflow.generate_crystal_freq_input()
print("CRYSTAL FREQCALC input:")
print(freq_input)
print()

# Generate CRYSTAL SCELPHONO input (phonon dispersion)
disp_input = workflow.generate_crystal_dispersion_input()
print("CRYSTAL SCELPHONO input:")
print(disp_input)


## 10. Thermal Properties

After phonon calculations complete, thermal properties can be computed.

In [ ]:
# Example: Compute zero-point energy from frequencies
frequencies_cm = [150.2, 220.5, 320.8, 450.1, 520.3, 680.7]  # cm^-1

zpe = workflow.compute_zero_point_energy(frequencies_cm)
print(f"Zero-point energy: {zpe:.4f} eV")

# Analyze Gamma-point frequencies for imaginary modes
workflow.analyze_gamma_frequencies(frequencies_cm)
print(f"\nHas imaginary modes: {workflow.result.has_imaginary}")
print(f"Minimum frequency: {workflow.result.min_frequency:.2f} cm^-1")


## 11. Combining Templates with Workflows

End-to-end example: Use a template for the SCF job, then run a phonon workflow.

In [ ]:
# End-to-end: Use template for SCF, then run phonon workflow
ctrl = CrystalController(db_path="phonon_demo.db")

# Step 1: Browse templates for a suitable starting point
print("Step 1: Browse basic templates")
for t in list_templates(category="basic"):
    print(f"  {t.category}/{t.name}")

# Step 2: Submit SCF job (in practice, use actual template content)
print("\nStep 2: Submit SCF job")
scf_job = JobSubmission(
    name="si_scf_for_phonon",
    dft_code=DftCode.CRYSTAL,
    input_content="... SCF input from template ...",
)
scf_pk = ctrl.submit_job(scf_job)
print(f"  SCF job PK={scf_pk}")

# Step 3: Configure phonon workflow from SCF result
print("\nStep 3: Configure phonon workflow")
phonon_config = PhononConfig(
    source_job_pk=scf_pk,
    method=PhononMethod.PHONOPY,
    supercell_dim=[2, 2, 2],
)
phonon_wf = PhononWorkflow(phonon_config)
print("  Phonon workflow ready")

# Step 4: The workflow generates displacement structures
# and creates jobs for each displacement
print("\nStep 4: Ready to generate displacements and submit jobs")
print("  (In production: extract structure from SCF job, generate displacements)")


## 12. Workflow Serialization

Workflows can be serialized to JSON for storage and restart.

In [ ]:
# Serialize workflow to JSON
json_str = workflow.to_json()
print("Serialized workflow:")
print(json_str[:500] + "...")

# Deserialize from JSON
restored_workflow = PhononWorkflow.from_json(json_str)
print(f"\nRestored workflow:")
print(f"  Method: {restored_workflow.config.method.value}")
print(f"  Supercell: {restored_workflow.config.supercell_dim}")
print(f"  Status: {restored_workflow.result.status}")


## Summary

In this notebook, you learned how to:

1. **Browse and filter** DFT templates by category and code
2. **Load templates** for CRYSTAL, VASP, and Quantum Espresso
3. **Configure phonon workflows** with phonopy or native methods
4. **Generate displacement structures** for finite difference calculations
5. **Analyze phonon results** including frequencies and thermal properties
6. **Combine templates and workflows** for end-to-end calculations
7. **Serialize/deserialize workflows** for persistence

**Next Steps:**
- Explore other workflow types (band structure, DOS, convergence)
- Create custom templates for your research
- Chain multiple workflows together (relax → phonon → thermal analysis)
- Use the Materials Project API to fetch structures (see `02_materials_project.ipynb`)